<a href="https://colab.research.google.com/github/FilipeSquire/WebScrappping/blob/main/VivaReal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
#Determina url para requisicao
#Estrutura de bairro é:#onde=BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Oeste-Bairro, ....
#Adicionar filtros: &nome=x ex: quartos=2
#Adicionar filtro preco: preco-ate=4000&preco-total=sim

#url = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/zona-leste/pinheiros/apartamento_residencial/'+'#area-desde=40'+'&com=mobiliado'+'&onde=BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Oeste-Pinheiros,'+'BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Oeste-Vila_Madalena,'+'BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Oeste-Sumare,'+'BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Oeste-Sumarezinho,'+'BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Oeste-Perdizes,'+'BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Oeste-Alto_de_Pinheiros,'+'BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Tatuape,'+'BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Vila_Gomes_Cardim,'+'BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Jardim_Analia_Franco'+'&preco-ate=2600'+'&preco-total=sim'

url = 'https://www.vivareal.com.br/aluguel/sp/sao-paulo/zona-leste/tatuape/apartamento_residencial/#onde=BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Tatuape,BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Vila_Gomes_Cardim,BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Vila_Carrao,BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Jardim_Analia_Franco&preco-ate=3100'
#Faz requisicao, passando numero da pagina como 1
doc = requests.get(url.format(n_pagina=1))

In [ ]:
url

'https://www.vivareal.com.br/aluguel/sp/sao-paulo/zona-leste/tatuape/apartamento_residencial/#onde=BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Tatuape,BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Vila_Gomes_Cardim,BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Vila_Carrao,BR-Sao_Paulo-NULL-Sao_Paulo-Zona_Leste-Jardim_Analia_Franco&preco-ate=3100'

In [ ]:
#Utiliza o BeautifulSoup para interpetar o conteudo de doc como HTML
parsed_doc = BeautifulSoup(doc.content, 'html.parser')

# Encontra todas as divs cuja classe é js-card-selector e traz como lista
cards = parsed_doc.find_all('div', class_='js-card-selector')
# A quantidade de cartoes na pagina é igual ao tamanho da lista


#ap recebe o primeiro card da lista paa desenvolvermos a logica
ap = cards[0]

# Name recebe o conteudo do elemento <span class=property-card__title>
# Aplicamos o metodo strip para remover espaços excedentes no começo e no final do texto

name = ap.find('span', class_='property-card__title').text.strip()
name

IndexError: list index out of range

In [ ]:
parsed_doc.find_all('div', class_='js-card-selector')

[]

In [ ]:
'vivareal.com.br' + ap.find('a', class_='property-card__labels-container').get('href')

'vivareal.com.br/imovel/apartamento-3-quartos-tatuape-zona-leste-sao-paulo-com-garagem-92m2-aluguel-RS4500-id-2612552075/'

In [ ]:
#Rodando para todos os cartoes das 20 primeiras paginas

result = []

for pag in range(1,38
                 ):
  print(f'Esto na pagina {pag}')
  doc = requests.get(url.format(n_pagina=pag))
  parsed_doc = BeautifulSoup(doc.content, 'html.parser')
  cards = parsed_doc.find_all('div', class_='js-card-selector')

  for ap in cards:
    #Extrai o id da div na hierarquia acima da atual
    ap_id = int(ap.parent.get('id'))
    #Extrai o titulo do anuncio
    name = ap.find('span', class_='property-card__title').text.strip()
    #Extrai endereço
    address = ap.find('span', class_='property-card__address').text.strip()
    #Extrai link
    link = 'vivareal.com.br' + ap.find('a', class_='property-card__labels-container').get('href')
    #Extrai area do ap
    area = int(ap.find('span', class_='property-card__detail-area').text.strip())
    #Extrai numero de quartos
    try:
      rooms = int(
          ap
          .find('li', class_='property-card__detail-room')
          .find('span', class_='property-card__detail-value')
          .text
          .strip()
      )
    except ValueError:
      rooms = 0

    #Extrai numero de banheiros
    try:
      bathrooms = int(
        ap
        .find('li', class_='property-card__detail-bathroom')
        .find('span', class_='property-card__detail-value')
        .text
        .strip()
      )
    except ValueError:
      bathrooms = 0

    #Se garagem for numero, extrair numero de vagas de garagem
    try:
      garages = int(
          ap
          .find('li', class_='property-card__detail-garage')
          .find('span', class_= 'property-card__detail-value')
          .text
          .strip()
      )
    except ValueError:
        #caso contrario define numero de garagens como sendo 0
        garages = 0

    #Extrai valor de aluguel
    rent_value = int(
        ap
       .find('div', class_='property-card__price')
       .find('p')
       .contents[0]
       .strip()
       .replace('R$ ','')
       .replace('.','')
    )

    #Consolida resultados em um dict
    result_dict = {
        'id': ap_id,
        'title':name,
        'address':address,
        'area':area,
        'num_rooms':rooms,
        'num_baths':bathrooms,
        'num_garages':garages,
        'rent_value':rent_value,
        'link':link
    }

    result.append(result_dict)
    sleep(1)

Esto na pagina 1
Esto na pagina 2
Esto na pagina 3
Esto na pagina 4
Esto na pagina 5
Esto na pagina 6
Esto na pagina 7
Esto na pagina 8
Esto na pagina 9
Esto na pagina 10
Esto na pagina 11
Esto na pagina 12
Esto na pagina 13
Esto na pagina 14
Esto na pagina 15
Esto na pagina 16
Esto na pagina 17
Esto na pagina 18
Esto na pagina 19
Esto na pagina 20
Esto na pagina 21
Esto na pagina 22
Esto na pagina 23
Esto na pagina 24
Esto na pagina 25
Esto na pagina 26
Esto na pagina 27
Esto na pagina 28
Esto na pagina 29
Esto na pagina 30
Esto na pagina 31
Esto na pagina 32
Esto na pagina 33
Esto na pagina 34
Esto na pagina 35
Esto na pagina 36
Esto na pagina 37


In [ ]:
df = pd.DataFrame(result)
df

,id,title,address,area,num_rooms,num_baths,num_garages,rent_value,link
0,2612552075,"Apartamento com 3 Quartos para Aluguel, 92m²","Rua Ivaí, 277 - Tatuapé, São Paulo - SP",92,3,3,2,4500,vivareal.com.br/imovel/apartamento-3-quartos-t...
1,2611994784,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2300,vivareal.com.br/imovel/apartamento-2-quartos-t...
2,2607384182,"Apartamento com 2 Quartos para Aluguel, 50m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",50,2,1,1,1800,vivareal.com.br/imovel/apartamento-2-quartos-t...
3,2608783977,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,2,2300,vivareal.com.br/imovel/apartamento-2-quartos-t...
4,2611964393,"Apartamento com 2 Quartos para Aluguel, 57m²","Rua Arnaldo Cintra, 190 - Tatuapé, São Paulo - SP",57,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...
...,...,...,...,...,...,...,...,...,...
1327,2601660677,"Apartamento com 3 Quartos para Aluguel, 165m²","Rua Tuiuti, 11 - Tatuapé, São Paulo - SP",165,3,4,3,11000,vivareal.com.br/imovel/apartamento-3-quartos-t...
1328,2612829707,"Apartamento com Quarto para Aluguel, 35m²","Rua Ulisses Cruz, 220 - Tatuapé, São Paulo - SP",35,1,1,0,1800,vivareal.com.br/imovel/apartamento-1-quartos-t...
1329,2609970762,"Apartamento com 3 Quartos para Aluguel, 95m²","Rua Serra de Bragança, 855 - Tatuapé, São Paul...",95,3,2,2,4000,vivareal.com.br/imovel/apartamento-3-quartos-t...
1330,2598634960,Apartamento com 2 Quartos à Venda/Aluguel 115m²,"Rua Professor Pedreira de Freitas, 900 - Tatua...",115,2,2,3,9000,vivareal.com.br/imovel/apartamento-2-quartos-t...


#Getting IPTU

In [ ]:
df['link']

0       vivareal.com.br/imovel/apartamento-3-quartos-t...
1       vivareal.com.br/imovel/apartamento-2-quartos-t...
2       vivareal.com.br/imovel/apartamento-2-quartos-t...
3       vivareal.com.br/imovel/apartamento-2-quartos-t...
4       vivareal.com.br/imovel/apartamento-2-quartos-t...
                              ...                        
1327    vivareal.com.br/imovel/apartamento-3-quartos-t...
1328    vivareal.com.br/imovel/apartamento-1-quartos-t...
1329    vivareal.com.br/imovel/apartamento-3-quartos-t...
1330    vivareal.com.br/imovel/apartamento-2-quartos-t...
1331    vivareal.com.br/imovel/apartamento-2-quartos-t...
Name: link, Length: 1332, dtype: object

In [ ]:
df['link'].iloc[1331]

'vivareal.com.br/imovel/apartamento-2-quartos-tatuape-zona-leste-sao-paulo-com-garagem-69m2-aluguel-RS2500-id-2614576424/'

In [ ]:
url = 'http://www.' + df['link'].iloc[0]

  #Faz requisicao, passando numero da pagina como 1
doc = requests.get(url)
 #Utiliza o BeautifulSoup para interpetar o conteudo de doc como HTML
parsed_doc = BeautifulSoup(doc.content, 'html.parser')


parsed_doc

 <!DOCTYPE html>
 <html lang="pt-BR" prefix="og: https://ogp.me/ns#"> <head> <meta content="5DB2670D7BB1D1D3E36C46F7C3D59380" name="msvalidate.01"/> <meta content="omni2c019e0" name="omniverify"/> <meta content="https://www.vivareal.com.br/imovel/apartamento-3-quartos-tatuape-zona-leste-sao-paulo-com-garagem-92m2-aluguel-RS4500-id-2612552075/" property="og:url"/> <meta content="website" property="og:type"/> <meta content="https://resizedimgs.vivareal.com/fit-in/1600x720/named.images.sp/292f2e13093e343aca607e3216778f8d/image.jpg" property="og:image"/> <meta content="image/jpeg" property="og:image:type"/> <meta content="https://resizedimgs.vivareal.com/fit-in/1600x720/named.images.sp/292f2e13093e343aca607e3216778f8d/image.jpg" property="og:image:secure_url"/> <meta content="Apartamento na Rua Ivaí, 277, Tatuapé em São Paulo, por R$ 4.500/Mês - Viva Real" property="og:title"/> <meta content="Alugue Apartamento com 3 Quartos e 92 m² por R$ 4.500/Mês na Rua Ivaí, 277 - Tatuapé - São Paulo -

In [ ]:
parsed_doc.find('span', class_='price__list-value iptu js-iptu').text.strip()

'R$ 300'

In [ ]:
teste = df.copy()
col = []

for x in range(0, teste.shape[0]):
  url = 'http://www.' + teste['link'].iloc[x]

  #Faz requisicao, passando numero da pagina como 1
  doc = requests.get(url)
  #Utiliza o BeautifulSoup para interpetar o conteudo de doc como HTML
  parsed_doc = BeautifulSoup(doc.content, 'html.parser')

  try:
    iptu = parsed_doc.find('span', class_='price__list-value iptu js-iptu')

    if iptu != None:
      col.append(iptu.text.strip())
    else:
      col.append(' ')
  except ValueError:
    col.append(' ')

col
teste['iptu'] = pd.Series(col)

In [ ]:
df['iptu'] = teste['iptu']

In [ ]:
#Condominio
teste = df.copy()
col = []

for x in range(0, teste.shape[0]):
  url = 'http://www.' + teste['link'].iloc[x]

  #Faz requisicao, passando numero da pagina como 1
  doc = requests.get(url)
  #Utiliza o BeautifulSoup para interpetar o conteudo de doc como HTML
  parsed_doc = BeautifulSoup(doc.content, 'html.parser')

  try:
    iptu = parsed_doc.find('span', class_='price__list-value condominium js-condominium')

    if iptu != None:
      col.append(iptu.text.strip())
    else:
      col.append(' ')
  except ValueError:
    col.append(' ')

col
teste['condo'] = pd.Series(col)

In [ ]:
df['condo'] = teste['condo']

In [ ]:
#Condominio + rent
teste = df.copy()
col = []

for x in range(0, teste.shape[0]):
  url = 'http://www.' + teste['link'].iloc[x]

  #Faz requisicao, passando numero da pagina como 1
  doc = requests.get(url)
  #Utiliza o BeautifulSoup para interpetar o conteudo de doc como HTML
  parsed_doc = BeautifulSoup(doc.content, 'html.parser')

  try:
    iptu = parsed_doc.find('span', class_='price__list-value rent-condominium js-total-rental-price')

    if iptu != None:
      col.append(iptu.text.strip())
    else:
      col.append(' ')
  except ValueError:
    col.append(' ')

col
teste['total'] = pd.Series(col)

In [ ]:
df['total'] = teste['total']
df.query('id == 2605050588')

,id,title,address,area,num_rooms,num_baths,num_garages,rent_value,link,iptu,condo,total
9,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,R$ 360,R$ 2.760
45,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,,
81,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,,
117,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,,
153,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,,
189,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,,
225,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,,
261,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,,
297,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,,
333,2605050588,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,,,


#**Dump**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
gc = gspread.authorize(GoogleCredentials.get_application_default())

TypeError: ignored

In [ ]:
from google.colab import auth
import gspread
from google.auth import default
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


#defining my worksheet
worksheet = gc.open('Real State - Leste').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '19QXCyoX8SlnzVG3-CcVHoXgSdTvLOmG08T2v1UzKTQs',
 'updatedRange': 'Data!A1:K1333',
 'updatedRows': 1333,
 'updatedColumns': 11,
 'updatedCells': 14663}

In [ ]:
df.to_json()

'{"id":{"0":2539909473,"1":2575978265,"2":2588468192,"3":2576858014,"4":2588186307,"5":2585124942,"6":2586736270,"7":2577598122,"8":2583517919,"9":2584758469,"10":2575612429,"11":2585717848,"12":2575702618,"13":2575543624,"14":2586790060,"15":2578431042,"16":2585022196,"17":2578572844,"18":2579596458,"19":2584605575,"20":2586278851,"21":2575613130,"22":2588464324,"23":2586152945,"24":2586394733,"25":2543940592,"26":2582443906,"27":2581468148,"28":2586207083,"29":2547755468,"30":2576086273,"31":2570014893,"32":2574543831,"33":2579578654,"34":2579562270,"35":2557723597,"36":2539909473,"37":2575978265,"38":2588468192,"39":2576858014,"40":2588186307,"41":2585124942,"42":2586736270,"43":2577598122,"44":2583517919,"45":2584758469,"46":2575612429,"47":2585717848,"48":2575702618,"49":2575543624,"50":2586790060,"51":2578431042,"52":2585022196,"53":2578572844,"54":2579596458,"55":2584605575,"56":2586278851,"57":2575613130,"58":2588464324,"59":2586152945,"60":2586394733,"61":2543940592,"62":25824

In [ ]:
df

,id,title,address,area,num_rooms,num_baths,num_garages,rent_value,link,iptu
0,2612552075,"Apartamento com 3 Quartos para Aluguel, 92m²","Rua Ivaí, 277 - Tatuapé, São Paulo - SP",92,3,3,2,4500,vivareal.com.br/imovel/apartamento-3-quartos-t...,R$ 300
1,2611994784,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,1,2300,vivareal.com.br/imovel/apartamento-2-quartos-t...,R$ 90
2,2607384182,"Apartamento com 2 Quartos para Aluguel, 50m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",50,2,1,1,1800,vivareal.com.br/imovel/apartamento-2-quartos-t...,R$ 60
3,2608783977,"Apartamento com 2 Quartos para Aluguel, 52m²","Rua Arnaldo Cintra, 454 - Tatuapé, São Paulo - SP",52,2,1,2,2300,vivareal.com.br/imovel/apartamento-2-quartos-t...,R$ 90
4,2611964393,"Apartamento com 2 Quartos para Aluguel, 57m²","Rua Arnaldo Cintra, 190 - Tatuapé, São Paulo - SP",57,2,1,1,2400,vivareal.com.br/imovel/apartamento-2-quartos-t...,R$ 58
...,...,...,...,...,...,...,...,...,...,...
1327,2601660677,"Apartamento com 3 Quartos para Aluguel, 165m²","Rua Tuiuti, 11 - Tatuapé, São Paulo - SP",165,3,4,3,11000,vivareal.com.br/imovel/apartamento-3-quartos-t...,
1328,2612829707,"Apartamento com Quarto para Aluguel, 35m²","Rua Ulisses Cruz, 220 - Tatuapé, São Paulo - SP",35,1,1,0,1800,vivareal.com.br/imovel/apartamento-1-quartos-t...,
1329,2609970762,"Apartamento com 3 Quartos para Aluguel, 95m²","Rua Serra de Bragança, 855 - Tatuapé, São Paul...",95,3,2,2,4000,vivareal.com.br/imovel/apartamento-3-quartos-t...,
1330,2598634960,Apartamento com 2 Quartos à Venda/Aluguel 115m²,"Rua Professor Pedreira de Freitas, 900 - Tatua...",115,2,2,3,9000,vivareal.com.br/imovel/apartamento-2-quartos-t...,


In [ ]:
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1FONIoHHqHIUcGYgmvspRFbWnD67UYiEbPegSTVQQVrM')
sheets = gsheets.get_worksheet(1)

sheets.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1FONIoHHqHIUcGYgmvspRFbWnD67UYiEbPegSTVQQVrM',
 'updatedCells': 13330,
 'updatedColumns': 10,
 'updatedRange': 'VivaReal!A1:J1333',
 'updatedRows': 1333}